# 前后端传输数据的编码格式 （contentType）

主要研究post请求数据的编码格式：`urlencoded,formdata,json`

可以朝后端发送post请求的方式：
    
    - form表单
    - ajax请求

## form表单

默认的数据编码格式是urlencoded

数据格式：username=lxt&password=123

django后端针对符合urlencoded编码格式的数据都会自动帮你解析封装到request.POST中。

    username=lxt&password=123 >>> request.POST
    
如果把编码格式修改为multipart/formdata,那么针对普通的键值对还是解析到request.POST中，而将文件解析到request.FILES中

form表单是没有办法发送json格式数据的。

## ajax

默认编码格式是 urlencoded

数据格式：username=lxt&password=123

django后端针对符合urlencoded编码格式的数据都会自动帮你解析封装到request.POST中。

    username=lxt&password=123 >>> request.POST
    


# ajax发送json格式数据

request对象方法补充

    request.is_ajax() 判断当前请求是否是ajax请求，返回布尔值

前后端传输数据的时候，一定要确保编码格式跟数据真正的格式是一致的。

templates中html文件


```
<button class="btn btn-danger" id="d1">点击</button>
<script>
    $('d1').click(function () {
        $.ajax({
            url:'',
            type:'post',
            data:JSON.stringify({'username':'lxt', 'age':28}),
            contentType:'application/json'  # 指定编码格式
            success:function () {
                
            }
        })
    })
</script>
```

视图函数
```
if request.is_ajax():
    # 针对json格式数据需要自己手动处理
    json_bytes = request.body  # django 后端不会帮你处理json格式数据，需要自己去request.body中取
    json_str = json_bytes.decode('utf-8')
    json_dict = json.loads(json_str)
    
    # json.loads括号内如果传入了一个二进制格式的数据，那么内部会自动解码再反序列化
    json_dict = json.loads(json_bytes)
    
    print(json_dict, type(json_dict))  #{'username': 'lxt', 'age': 25} <class 'dict'>
```

# ajax 发送文件数据

```
<p>username:<input type="text" id="d1"></p>
<p>password:<input type="text" id="d2"></p>
<p><input type="file" id="d3"></p>
<button class="btn btn-info" id="d4">点击</button>

<script>
    // 点击按钮后朝后盾发送普通键值对和问价数据
    $('#d4').on('click', function () {
        // 1 需要先 利用FormData内置对象
        let formDataObj = new FormData()
        // 2 添加普通键值对
        formDataObj.append('username', $('#d1').val());
        formDataObj.append('password', $('#d2').val());
        // 3 添加文件对象
        formDataObj.append('myfile', $('#d3')[0].files[0])
        // 4 将对象基于ajax发送到后端
        $.ajax({
            url:'',
            type:'post',
            data:formDataObj,  // 直接将对象放在data后即可
            // ajax 发送文件必须要指定的两个参数
            contentType:false,  // 不需使用任何编码， django后端能够自动识别formdatta对象
            processData:false,  // 告诉浏览器不要对你的数据进行任何处理
            
            success:function (args) {
                
            }
        })
        
        })
</script>
```

```
if request.is_ajax():
    if request.method == "POST":
        print(request.POST)
        prtin(request.FILES)
        
# django 后端能够直接识别到formdata对象并且能够将内部的普通键值对自动解析并封装到request.POST中，文件对象自动解析并且封装到request.FILES中。
```

# django 自带的序列化组件

多用于前后端分离项目，主要就是写接口，写接口就是利用序列化组件渲染数据然后写一个接口文档

```
from django.core import serializers
user_queryset = models.User.objects.all()
res = serializers.serialize('json', user_queryset)
```

# ajax结合sweetalert

需自行整理

# 批量插入数据

当你想要批量插入数据的时候，使用orm提供的bulk_create能够大大的减少操作时间

```
boot_list = []
for i in range(10000):
    book_obj = model.Book(title="第{}本书".format(i))
    book_list.append(book_obj)
    
models.Book.objects.bulk_create(book_list)
```

# 分页器